# Mapping NIPA Data to a HRV 2021 Economy in ABGP
## Omar Licandro and Juan Ignacio Vizcaino 

In [ ]:
import os
os.chdir(r"C:\\Users\\Nacho\\Dropbox\\Omar&Nacho\\Baqaee_Burstein\\Note on Structural Change\\Code")

In [ ]:
import pandas as pd
import plotly as plt
import numpy as np
import requests
from io import StringIO
import plotly.graph_objects as go

In [ ]:
def apply_standard_plotly_style(fig, yaxis_range=None):
    fig.update_layout(
        font          = dict(family="Open Sans", size=14, color="black"),
        plot_bgcolor  = "white",
        paper_bgcolor = "white",
        legend        = dict(x=0.05, y=0.95, xanchor="left", yanchor="top",    
                        bgcolor="rgba(0,0,0,0)", bordercolor="rgba(0,0,0,0)"),
        margin=dict(l=90, r=20, t=30, b=40))
    
    fig.update_xaxes(
        showgrid=True, gridcolor = "lightgray",dtick = 5,  
        showline=True, linewidth = 1.0, linecolor="black", mirror=True,
        ticks="outside", tickfont= dict(family="Open Sans", size=14))
    
    fig.update_yaxes(
        showgrid=True, gridcolor="lightgray",
        tickformat=".1f",
        showline=True, linewidth=1.0, linecolor="black", mirror=True,
        ticks="outside", tickfont=dict(family="Open Sans", size=14),
        range=yaxis_range)
    
    return fig

In [ ]:
# Define a Function to Compute Fixed-Base Laspeyres and Paasche Quantity Indices
def Q_I_Fixed_Base(df, vars, base_year=None):
    """
    Compute a fixed-base quantity index for a group of variables, allowing for positive and negative contributions.

    Parameters:
        df:         DataFrame containing the data
        vars:       list of (variable name, sign) tuples, e.g. [("PCE", 1), ("GCE", 1), ("IM_GOOD", -1)]
        base_year:  the year to use as the base (default 1947)
        Returns:    A pandas Series with the index for all years
    """
    if base_year is None:
        raise ValueError("You must specify a base_year.")

    # Get base period prices and quantities for all variables
    p_base = [df.loc[df['year'] == base_year, f"{var}_P"].values[0] for var, _ in vars]
    q_base = [df.loc[df['year'] == base_year, f"{var}_Q"].values[0] for var, _ in vars]

    # Denominator is constant
    denom = sum(sign * p_base[j] * q_base[j] for j, (var, sign) in enumerate(vars))

    numerators = []
    for _, row in df.iterrows():
        num = sum(sign * p_base[j] * row[f"{var}_Q"] for j, (var, sign) in enumerate(vars))
        numerators.append(num)

    return pd.Series(np.array(numerators) / denom, index=df.index)

In [ ]:
# Define a Function to Compute Chained Laspeyres and Paasche Quantity Indices
def Q_I_Moving_Base(df, vars, index_type=None):
    """
    Compute chained Laspeyres or Paasche quantity index for a group of variables using a moving base (previous year),
    allowing for positive and negative contributions.

    Parameters:
        df:         DataFrame containing the data (must be sorted by year)
        vars:       list of (variable name, sign) tuples, e.g. [("PCE", 1), ("GCE", 1), ("IM_GOOD", -1)]
        index_type: "laspeyres" or "paasche"
    Returns:
        A pandas Series with the chained index for all years (first year is 1)
    """
    if index_type not in ["laspeyres", "paasche"]:
        raise ValueError("index_type must be 'laspeyres' or 'paasche'")

    ratios = []
    for i in range(1, len(df)):
        row = df.iloc[i]
        prev_row = df.iloc[i-1]
        if index_type == "laspeyres":
            num   = sum(sign * prev_row[f"{var}_P"] * row[f"{var}_Q"] for (var, sign) in vars)
            denom = sum(sign * prev_row[f"{var}_P"] * prev_row[f"{var}_Q"] for (var, sign) in vars)
        elif index_type == "paasche":
            num   = sum(sign * row[f"{var}_P"] * row[f"{var}_Q"] for (var, sign) in vars)
            denom = sum(sign * row[f"{var}_P"] * prev_row[f"{var}_Q"] for (var, sign) in vars)
        ratios.append(num / denom)
        
    # First period is normalized to 1, then chain the ratios
    chained_index = np.concatenate([[1], np.cumprod(ratios)])
    
    return pd.Series(chained_index, index=df.index)

### Download and Clean the Data

In [ ]:
# Download and load the SeriesRegister file
url_series = "https://apps.bea.gov/national/Release/TXT/SeriesRegister.txt"
res_series = requests.get(url_series)
series_df  = pd.read_csv(StringIO(res_series.text), sep="\t", encoding="ISO-8859-1")
series_df.columns = [c.strip() for c in series_df.columns]  

# Download and load the NIPA data 
url_data = "https://apps.bea.gov/national/Release/TXT/NipaDataA.txt"
res_data = requests.get(url_data)
data_df  = pd.read_csv(StringIO(res_data.text), sep="\t", encoding="ISO-8859-1")

In [ ]:
# Clean and prepare the data

data_df         = data_df["%SeriesCode,Period,Value"].str.split(",", n=2, expand=True)
data_df.columns = ["SeriesCode", "Period", "Value"]

data_df["Value"] = pd.to_numeric(data_df["Value"].str.replace('"', '').str.replace(",", ""), errors="coerce")

series_df = series_df["%SeriesCode,SeriesLabel,MetricName,CalculationType,DefaultScale,TableId:LineNo,SeriesCodeParents"] \
	.str.split(",", n=6, expand=True)
series_df.columns = ["SeriesCode", "SeriesLabel", "MetricName", "CalculationType", "DefaultScale", "TableId:LineNo", "SeriesCodeParents"]

df = data_df.merge(series_df, on="SeriesCode", how="inner")

In [ ]:
# Keep selected SeriesCodes
selected_codes = [
    "A191RC", "A191RA", "DPCERC", "DPCERA", "A014RC", "A955RC", "A955RA", "A782RC",
    "B230RC", "DGDSRC", "DSERRC", "DSERRA", "DGDSRA", "A253RC", "A019RC", "A646RC",
    "A255RC", "B656RC", "B253RA", "A006RC", "B646RA", "B255RA", "B656RA", "W172RC", "W172RA"
]
df = df[df.SeriesCode.isin(selected_codes)]

# Rename and reshape 
df = df.rename(columns={"Period": "year"})
df = df.sort_values(["SeriesCode", "year"])

# Keep relevant columns only
df      = df[["year", "Value", "SeriesCode"]]
df_wide = df.pivot(index="year", columns="SeriesCode", values="Value").reset_index()

In [ ]:
# Rename columns
rename_dict = {
    "A191RA": "GDP_QI", 
    "A191RC": "GDP", 
    "A014RC": "INI", 
    "A955RC": "GCE", 
    "A955RA": "GCE_QI",
    "A782RC": "GIE", 
    "DPCERC": "PCE", 
    "DPCERA": "PCE_QI", 
    "DGDSRC": "PCE_GOOD",
    "DGDSRA": "PCE_GOOD_QI", 
    "DSERRC": "PCE_SERV", 
    "DSERRA": "PCE_SERV_QI",
    "A006RC": "PIE", 
    "A019RC": "NX", 
    "A253RC": "EX_GOOD", 
    "A646RC": "EX_SERV",
    "A255RC": "IM_GOOD", 
    "B656RC": "IM_SERV", 
    "B253RA": "EX_GOOD_QI",
    "B646RA": "EX_SERV_QI", 
    "B255RA": "IM_GOOD_QI", 
    "B656RA": "IM_SERV_QI",
    "W172RC": "X_TOT", 
    "W172RA": "X_TOT_QI", 
    "B230RC": "POP"
}
df_wide = df_wide.rename(columns=rename_dict)

In [ ]:
# Keep only years after 1947 and before 2024
df_wide['year'] = df_wide['year'].astype(int)
df_wide = df_wide[(df_wide.year > 1946) & (df_wide.year < 2024)].copy()

In [ ]:
for var in ["GDP", "X_TOT", "PCE", "GCE", "PCE_GOOD", "PCE_SERV", "EX_GOOD", "EX_SERV", "IM_GOOD", "IM_SERV"]:
    df_wide[f"{var}_QI"] = df_wide[f"{var}_QI"]/df_wide.loc[df_wide["year"] == 2017,f"{var}_QI"].values[0]
    df_wide[f"{var}_P"]  = (df_wide[var] / df_wide.loc[df_wide["year"] == 2017,f"{var}"].values[0]) / df_wide[f"{var}_QI"]
    df_wide[f"{var}_Q"]  = df_wide[f"{var}_QI"] * df_wide.loc[df_wide["year"] == 2017,f"{var}"].values[0]

## Compute Chained Laspeyres and Paasche Quantity Indices

### Goods Quantity Index

In [ ]:
vars_good = [("PCE_GOOD", 1), ("EX_GOOD", 1), ("IM_GOOD", -1)]

df_wide["C_GOOD_Paasche_Chained_QI"]   = Q_I_Moving_Base(df_wide,vars_good, index_type="paasche")
df_wide["C_GOOD_Laspeyres_Chained_QI"] = Q_I_Moving_Base(df_wide,vars_good, index_type="laspeyres")
df_wide["C_GOOD_Fischer_Chained_QI"]   = (df_wide["C_GOOD_Paasche_Chained_QI"] * df_wide["C_GOOD_Laspeyres_Chained_QI"]) ** 0.5

### Services Quantity Index

In [ ]:
#Services
vars_serv = [("PCE_SERV", 1), ("GCE", 1), ("EX_SERV", 1), ("IM_SERV", -1)]

df_wide["C_SERV_Paasche_Chained_QI"]   = Q_I_Moving_Base(df_wide,vars_serv, index_type="paasche")
df_wide["C_SERV_Laspeyres_Chained_QI"] = Q_I_Moving_Base(df_wide,vars_serv, index_type="laspeyres")
df_wide["C_SERV_Fischer_Chained_QI"]   = (df_wide["C_SERV_Paasche_Chained_QI"] * df_wide["C_SERV_Laspeyres_Chained_QI"]) ** 0.5

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x    = df_wide["year"], y=df_wide["C_GOOD_Fischer_Chained_QI"],
    mode = 'lines',
    line = dict(color='black', width=2, dash='dot'),
    name = "Goods",
))


fig.add_trace(go.Scatter(
    x    = df_wide["year"], y=df_wide["C_SERV_Fischer_Chained_QI"],
    mode = 'lines',
    line = dict(color='black', width=2, dash='dash'),
    name = "Services",
))

fig.update_layout(
    title        = "Fischer-Ideal Chained Quantity Indices",
    yaxis_title  = "Index<br>(1947=1)",
    template     = "none",
    width        = 800
)


apply_standard_plotly_style(fig)

fig.show()

## Compute Chained Laspeyres and Paasche Price Indices

In [ ]:
df_wide["C_GOOD"] = df_wide["PCE_GOOD"] + df_wide["EX_GOOD"] - df_wide["IM_GOOD"] + df_wide["INI"]
df_wide["C_SERV"] = df_wide["PCE_SERV"] + df_wide["EX_SERV"] - df_wide["IM_SERV"] + df_wide["GCE"]

df_wide["C_TOT"]  = df_wide["C_GOOD"] + df_wide["C_SERV"]

In [ ]:
df_wide["C_GOOD_Fischer_Chained_P"]  = (df_wide["C_GOOD"]/df_wide["C_GOOD"].iloc[0])/df_wide["C_GOOD_Fischer_Chained_QI"] 
df_wide["C_SERV_Fischer_Chained_P"]  = (df_wide["C_SERV"]/df_wide["C_SERV"].iloc[0])/df_wide["C_SERV_Fischer_Chained_QI"] 

In [ ]:
df_wide["C_GOOD_Fischer_Chained_Q"] = df_wide["C_GOOD_Fischer_Chained_QI"] * df_wide["C_GOOD"].iloc[0]
df_wide["C_SERV_Fischer_Chained_Q"] = df_wide["C_SERV_Fischer_Chained_QI"] * df_wide["C_SERV"].iloc[0]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x       = df_wide["year"], y=df_wide["C_GOOD_Fischer_Chained_P"],
    mode    = 'lines',
    name    = "Goods",
    line    = dict(color='black', width=2, dash='dot')
))

fig.add_trace(go.Scatter(
    x       = df_wide["year"], y=df_wide["C_SERV_Fischer_Chained_P"],
    mode    = 'lines',
    name    = "Services",
    line    = dict(color='black', width=2, dash='dash')
))

fig.update_layout(
    title           = "Fischer-Ideal Chained Price Indices",
    xaxis_title     = "Year",
    yaxis_title     = "Index<br>(1947=1)",
    template        = "none",
    width           = 800
)

apply_standard_plotly_style(fig)

fig.show()

In [ ]:
#Notice that these are technically Laspeyres Price indices. I use the name "Paasche" because this is the index I will input in the function 
df_wide["C_GOOD_Paasche_Chained_P"]  = (df_wide["C_GOOD"]/df_wide["C_GOOD"].iloc[0])/df_wide["C_GOOD_Paasche_Chained_QI"]
df_wide["C_SERV_Paasche_Chained_P"]  = (df_wide["C_SERV"]/df_wide["C_SERV"].iloc[0])/df_wide["C_SERV_Paasche_Chained_QI"]

In [ ]:
# Normalize to 1 in 1947
df_wide["C_GOOD_Paasche_Chained_P"]  = df_wide["C_GOOD_Paasche_Chained_P"] / df_wide.loc[df_wide["year"] == 1947,"C_GOOD_Paasche_Chained_P"].values[0]
df_wide["C_SERV_Paasche_Chained_P"]  = df_wide["C_SERV_Paasche_Chained_P"] / df_wide.loc[df_wide["year"] == 1947,"C_SERV_Paasche_Chained_P"].values[0]

In [ ]:
#Notice that these are technically Paasche Price indices. The same comment as above applies.

df_wide["C_GOOD_Laspeyres_Chained_P"]  = (df_wide["C_GOOD"]/df_wide["C_GOOD"].iloc[0])/df_wide["C_GOOD_Laspeyres_Chained_QI"]
df_wide["C_SERV_Laspeyres_Chained_P"]  = (df_wide["C_SERV"]/df_wide["C_SERV"].iloc[0])/df_wide["C_SERV_Laspeyres_Chained_QI"]

### Build Normalized Indices for Investment

In [ ]:
df_wide["X_TOT_P"]   = df_wide["X_TOT_P"] / df_wide.loc[df_wide["year"] == 1947 , "X_TOT_P"].values[0]
df_wide["X_TOT_QI"]  = df_wide["X_TOT_QI"] / df_wide.loc[df_wide["year"] == 1947, "X_TOT_QI"].values[0]
df_wide["X_TOT_Q"]   = df_wide["X_TOT_QI"] * df_wide["X_TOT"].iloc[0]

## Compute Paasche and Laspeyres Indices for Aggregate Consumption Expenditure and Investment

### Aggregate Consumption

In [ ]:
vars_C_TOT_QI_chained = [("GDP", 1), ("X_TOT", -1)]

df_wide["C_TOT_Paasche_Chained_QI"]   = Q_I_Moving_Base(df_wide,vars_C_TOT_QI_chained , index_type="paasche")
df_wide["C_TOT_Laspeyres_Chained_QI"] = Q_I_Moving_Base(df_wide,vars_C_TOT_QI_chained , index_type="laspeyres")
df_wide["C_TOT_Fischer_Chained_QI"]   = ( df_wide["C_TOT_Paasche_Chained_QI"] * df_wide["C_TOT_Laspeyres_Chained_QI"] ) ** 0.5
df_wide["C_TOT_Fischer_Chained_Q"]    = df_wide["C_TOT_Fischer_Chained_QI"] * df_wide["C_TOT"].iloc[0]

In [ ]:
df_wide["C_TOT_P"]                    = df_wide["C_TOT"]/df_wide["C_TOT_Fischer_Chained_QI"]
df_wide["C_TOT_P"]                    = df_wide["C_TOT_P"]/df_wide["C_TOT_P"].iloc[0]

## Plot the relative price of investment and the consumption expenditure share

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x       = df_wide["year"], y=df_wide["C_TOT_P"],
    mode    = 'lines',
    name    = "Aggregate Consumption",
    line    = dict(color='black', width=2, dash='dot')
))

fig.add_trace(go.Scatter(
    x       = df_wide["year"], y=df_wide["X_TOT_P"],
    mode    = 'lines',
    name    = "Aggregate Investment",
    line    = dict(color='black', width=2, dash='dash')
))

fig.update_layout(
    title           = "Fischer-Ideal Chained Price Indices",
    yaxis_title     = "Index<br>(1947=1)",
    template        = "none",
    width           = 800
)

apply_standard_plotly_style(fig)

fig.show()

In [ ]:
df_wide["X_Rel_P"] = df_wide["X_TOT_P"] / df_wide["C_TOT_P"]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x       = df_wide["year"], y=df_wide["X_Rel_P"],
    mode    = 'lines',
    name    = "Relative Price of Investment",
    line    = dict(color='black', width=2, dash='dot')
))


fig.update_layout(
    yaxis_title     = "Relative Price Index<br>(1947=1)",
    template        = "none",
    width=800
)

apply_standard_plotly_style(fig, yaxis_range=[0.5, 1.2])
fig.show()

### Share of Consumption in Total Expenditure

In [ ]:
df_wide["C_Share"]  = df_wide["C_TOT"] / ( df_wide["C_TOT"]+ df_wide["X_TOT"] )

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x       = df_wide["year"], y=df_wide["C_Share"],
    mode    = 'lines',
    name    = "Consumption Share",
    line    = dict(color='black', width=2, dash='solid')
))

fig.update_layout(
    template     = "none",
    width        = 800
)

apply_standard_plotly_style(fig, yaxis_range=[0.6, 1.0])
fig.show()

In [ ]:
np.average(df_wide["C_Share"]) 

## Build Fixed-Based Indices Aggregating From Fischer-Ideal Indices

Notice that these indices are built aggregatig Fisher-Ideal indices of consumption expenditures and invesment. In the next section we aggregate goods and services first, and total consumption later on, consistently, using the same fixed-base index than in the aggregate. 

In [ ]:
vars_E_chained          = [("C_GOOD_Fischer_Chained", 1), ("C_SERV_Fischer_Chained", 1)]

df_wide["E_FB_1947_QI"] = Q_I_Fixed_Base(df_wide,vars_E_chained, base_year=1947)
df_wide["E_FB_2023_QI"] = Q_I_Fixed_Base(df_wide,vars_E_chained, base_year=2023)
df_wide["E_FB_2023_QI"] = df_wide["E_FB_2023_QI"]/(df_wide["E_FB_2023_QI"].iloc[0])

df_wide["E_FB_1947_P"]  = (df_wide["C_TOT"]/df_wide["C_TOT"].iloc[0])/df_wide["E_FB_1947_QI"]
df_wide["E_FB_2023_P"]  = (df_wide["C_TOT"]/df_wide["C_TOT"].iloc[0])/df_wide["E_FB_2023_QI"]

df_wide["E_FB_1947_Q"]  = df_wide["E_FB_1947_QI"] * df_wide["C_TOT"].iloc[0]
df_wide["E_FB_2023_Q"]  = df_wide["E_FB_2023_QI"] * df_wide["C_TOT"].iloc[0]

df_wide["X_TOT_1947_Q"]  = df_wide["X_TOT_Q"]
df_wide["X_TOT_1947_QI"] = df_wide["X_TOT_QI"]
df_wide["X_TOT_1947_P"]  = df_wide["X_TOT_P"]

df_wide["X_TOT_2023_Q"]  = df_wide["X_TOT_Q"]
df_wide["X_TOT_2023_QI"] = df_wide["X_TOT_QI"]
df_wide["X_TOT_2023_P"]  = df_wide["X_TOT_P"]

In [ ]:
vars_Y_1947             = [("E_FB_1947", 1), ("X_TOT_1947", 1)]
df_wide["Y_FB_1947_QI"] = Q_I_Fixed_Base(df_wide,vars_Y_1947, base_year=1947)

In [ ]:
vars_Y_2023= [("E_FB_2023", 1), ("X_TOT_2023", 1)]
df_wide["Y_FB_2023_QI"] = Q_I_Fixed_Base(df_wide,vars_Y_2023, base_year=2023)
df_wide["Y_FB_2023_QI"] = df_wide["Y_FB_2023_QI"] / df_wide["Y_FB_2023_QI"].iloc[0]

In [ ]:
df_wide["Y_Fischer_Chained_QI"]   = ( df_wide["Y_FB_1947_QI"]*df_wide["Y_FB_2023_QI"] ) ** 0.5

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_wide["year"], y=np.log(df_wide["Y_Fischer_Chained_QI"]),
    mode='lines',
    name="Fischer-Ideal Quatity Index",
    line=dict(color='black', width=2, dash='solid')
))

fig.add_trace(go.Scatter(
    x=df_wide["year"], y=np.log(df_wide["Y_FB_2023_QI"]),
    name="Fixed-Base Quantity Index (2023)",
    mode='lines',
    line=dict(color='black', width=2, dash='dash')
))

fig.add_trace(go.Scatter(
    x=df_wide["year"], y=np.log(df_wide["Y_FB_1947_QI"]),
    mode='lines',
    name="Fixed-Base Quantity Index (1947)",
    line=dict(color='black', width=2, dash='dot')
))

fig.update_layout(
    title        = "GDP Indices",
    yaxis_title  = "Index<br>(Log scale;1947=0)",
    template     = "none",
    width        = 800
)

apply_standard_plotly_style(fig)

fig.show()


## Build Fixed-Based Indices Aggregating From Fixed-Based Indices

Notice that these indices are built aggregatig the corresponding fixed-based indices of consumption expenditures and invesment. In the next section we aggregate goods and services first, and total consumption later on, consistently, using the same fixed-base index than in the aggregate. 

In [ ]:
vars_good = [("PCE_GOOD", 1), ("EX_GOOD", 1), ("IM_GOOD", -1)]

df_wide["C_GOOD_FB_1947_QI"] = Q_I_Fixed_Base(df_wide,vars_good, base_year=1947)
df_wide["C_GOOD_FB_2023_QI"] = Q_I_Fixed_Base(df_wide,vars_good, base_year=2023)
df_wide["C_GOOD_FB_2023_QI"] = df_wide["C_GOOD_FB_2023_QI"] / df_wide["C_GOOD_FB_2023_QI"].iloc[0]

In [ ]:
vars_serv = [("PCE_SERV", 1), ("GCE", 1), ("EX_SERV", 1), ("IM_SERV", -1)]

df_wide["C_SERV_FB_1947_QI"] = Q_I_Fixed_Base(df_wide,vars_serv, base_year=1947)
df_wide["C_SERV_FB_2023_QI"] = Q_I_Fixed_Base(df_wide,vars_serv, base_year=2023)
df_wide["C_SERV_FB_2023_QI"] = df_wide["C_SERV_FB_2023_QI"] / df_wide["C_SERV_FB_2023_QI"].iloc[0]

In [ ]:
df_wide["C_GOOD_FB_1947_P"]  = (df_wide["C_GOOD"]/df_wide["C_GOOD"].iloc[0])/df_wide["C_GOOD_FB_1947_QI"]
df_wide["C_SERV_FB_1947_P"]  = (df_wide["C_SERV"]/df_wide["C_SERV"].iloc[0])/df_wide["C_SERV_FB_1947_QI"]

In [ ]:
df_wide["C_GOOD_FB_2023_P"]  = (df_wide["C_GOOD"]/df_wide["C_GOOD"].iloc[0])/df_wide["C_GOOD_FB_2023_QI"]
df_wide["C_SERV_FB_2023_P"]  = (df_wide["C_SERV"]/df_wide["C_SERV"].iloc[0])/df_wide["C_SERV_FB_2023_QI"]

In [ ]:
df_wide["C_GOOD_FB_1947_Q"] = df_wide["C_GOOD_FB_1947_QI"] * df_wide["C_GOOD"].iloc[0]
df_wide["C_SERV_FB_1947_Q"] = df_wide["C_SERV_FB_1947_QI"] * df_wide["C_SERV"].iloc[0]

df_wide["C_GOOD_FB_2023_Q"] = df_wide["C_GOOD_FB_2023_QI"] * df_wide["C_GOOD"].iloc[0]
df_wide["C_SERV_FB_2023_Q"] = df_wide["C_SERV_FB_2023_QI"] * df_wide["C_SERV"].iloc[0]

### Aggregate Into a Final Fixed-Based Index
Having built the fixed-base indices for goods and services, now we proceed to aggregate them into a final, fixed-base, GDP index.

In [ ]:
vars_E_1947 = [("C_GOOD_FB_1947", 1), ("C_SERV_FB_1947", 1)]
vars_E_2023 = [("C_GOOD_FB_2023", 1), ("C_SERV_FB_2023", 1)]

df_wide["E_FB_1947_QI"] = Q_I_Fixed_Base(df_wide,vars_E_1947, base_year=1947)
df_wide["E_FB_2023_QI"] = Q_I_Fixed_Base(df_wide,vars_E_2023, base_year=2023)
df_wide["E_FB_2023_QI"] = df_wide["E_FB_2023_QI"]/df_wide["E_FB_2023_QI"].iloc[0]

df_wide["E_FB_Fischer_QI"] = (df_wide["E_FB_1947_QI"] * df_wide["E_FB_2023_QI"]) ** 0.5

In [ ]:
df_wide["E_FB_1947_Q"] = df_wide["E_FB_1947_QI"] * df_wide["C_TOT"].iloc[0]
df_wide["E_FB_2023_Q"] = df_wide["E_FB_2023_QI"] * df_wide["C_TOT"].iloc[0] 

In [ ]:
vars_Y_1947 = [("E_FB_1947", 1), ("X_TOT", 1)]
vars_Y_2023 = [("E_FB_2023", 1), ("X_TOT", 1)]

df_wide["Y_FB_1947_QI"] = Q_I_Fixed_Base(df_wide,vars_Y_1947, base_year=1947)
df_wide["Y_FB_2023_QI"] = Q_I_Fixed_Base(df_wide,vars_Y_2023, base_year=2023)
df_wide["Y_FB_2023_QI"] = df_wide["Y_FB_2023_QI"]/df_wide["Y_FB_2023_QI"].iloc[0]

In [ ]:
df_wide["Y_FB_Fischer_QI"] = (df_wide["Y_FB_1947_QI"] * df_wide["Y_FB_2023_QI"]) ** 0.5

In [ ]:
fig = go.Figure()

# Fischer Fixed-Base: solid black
fig.add_trace(go.Scatter(
    x    = df_wide["year"], y=np.log(df_wide["Y_FB_Fischer_QI"]),
    mode = 'lines',
    name = "Fischer-Ideal Index",
    line = dict(color='black', width=2, dash='solid')
))

# 2023 Fixed-Base: dashed black
fig.add_trace(go.Scatter(
    x    = df_wide["year"], y=np.log(df_wide["Y_FB_2023_QI"]),
    mode = 'lines',
    name = "Fixed-Base Index (2023)",
    line = dict(color='black', width=2, dash='dash')
))

# 1947 Fixed-Base: dotted black
fig.add_trace(go.Scatter(
    x    = df_wide["year"], y=np.log(df_wide["Y_FB_1947_QI"]),
    mode = 'lines',
    name = "Fixed-Base Index (1947)",
    line = dict(color='black', width=2, dash='dot')
))

fig.update_layout(
    yaxis_title = "Quantity Index<br>(Log scale; 1947=0)",
    template    = "none",
    width       = 800
)

apply_standard_plotly_style(fig, yaxis_range=[0, None])

fig.show()

In [ ]:
#fig.write_image("Figures/real_GDP.pdf", format="pdf", width=1200, height=800, scale=2)

In [ ]:
# Filter data from 1980 onward
df_wide_1980 = df_wide[df_wide["year"] >= 1980].copy()

# Normalize indices so that log(index) = 0 in 1980
for col in ["Y_FB_Fischer_QI", "Y_FB_2023_QI", "Y_FB_1947_QI"]:
    df_wide_1980[col] = df_wide_1980[col] / df_wide_1980.loc[df_wide_1980["year"] == 1980, col].values[0]

fig = go.Figure()

# Fischer Fixed-Base: solid black
fig.add_trace(go.Scatter(
    x    = df_wide_1980["year"], y=np.log(df_wide_1980["Y_FB_Fischer_QI"]),
    mode = 'lines',
    name = "Fischer-Ideal Index",
    line = dict(color='black', width=2, dash='solid')
))

# 2023 Fixed-Base: dashed black
fig.add_trace(go.Scatter(
    x    = df_wide_1980["year"], y=np.log(df_wide_1980["Y_FB_2023_QI"]),
    mode = 'lines',
    name = "Fixed-Base Index (2023)",
    line = dict(color='black', width=2, dash='dash')
))

# 1947 Fixed-Base: dotted black
fig.add_trace(go.Scatter(
    x    = df_wide_1980["year"], y=np.log(df_wide_1980["Y_FB_1947_QI"]),
    mode = 'lines',
    name = "Fixed-Base Index (1947)",
    line = dict(color='black', width=2, dash='dot')
))

fig.update_layout(
    yaxis_title = "Quantity Index<br>(Log Scale; 1980=0)",
    template    = "none",
    width       = 800
)
apply_standard_plotly_style(fig, yaxis_range=[0, None])

fig.show()
